# Решение краевыx задач. Методы коллокаций, наименьших квадратов и Галеркина
## Задача
$$
\begin{cases}  
u(x)y''+ p(x)y=-1,     &x\in[a,b],
\\
y(a) = A,
\\
y(b) = B
\end{cases}  
$$

### Определения
- Граничные условия: 
$$
\begin{cases}  
y(a) = A,
\\
y(b) = B
\end{cases}  
$$
- Однородные граничные условия:
$$
\begin{cases}  
y(a) = 0,
\\
y(b) = 0
\end{cases}  
$$

### Базисная система
$j_{0}(x),j_{1}(x),...j_{n}(x)$ - базисная система, если:
1. $j_{i}(x), i\in[0,n]$ дважды непрерывно дифференцируемы
2. $j_{0}(x)$ удовлетворяет граничному условию
3. функции $j_{i}(x), i\in[1,n]$ - линейно независимы на $[a, b]$ и удовлетворяют однородным граничным условиям


### Вид решения
Представим решение в виде линейной комбинации базисных функций
$$
y_{n} = j_{0}(x) + a_{1}j_{1}(x) + ... + a_{n-1}j_{n-1}(x)
$$

### Невязка
$$
Y(x, a_{1}, ..., a_{n-1}) = u(x)y''+ p(x)y + 1
$$

# Частный случай
### Задача
$$
\begin{cases}  
u(x)y''+ p(x)y=-1,     &x\in[-1,1],
\\
y(-1) = 0,
\\
y(1) = 0
\end{cases}  
$$

### Базис
Возьмем базис:
$$
\begin{cases}
j_{0}(x)=0,
\\
j_{i}(x)=x^i(1-x),     &i=1,2,...
\end{cases}  
$$

### Вид решения
Расчитаем производные.

$$
\begin{cases}
y_{n} = \sum_{i=1}^{n-1}a_{i}(x^{i}-x^{i+1})
\\
y_{n}' = \sum_{i=1}^{n-1}a_{i}(ix^{i-1}-(i+1)x^{i})
\\
y_{n}'' = -2a_{1} + \sum_{i=2}^{n-1}a_{i}((i-1)ix^{i-2}-i(i+1)x^{i-1})
\end{cases}
$$

### Невязка
$$
Y(x, a_{1}, ..., a_{n-1}) = a_{1}(-2u(x)+ x(1-x)p(x)) + \sum_{i=2}^{n-1}a_{i}(u(x)x^{i-2}((i-1)i-i(i+1)x) + p(x)x^{i}(1-x)) + 1
$$

In [78]:
import sympy
import numpy
import matplotlib.pyplot as plt
from functools import reduce
from prettytable import PrettyTable

In [170]:
def u(x):
    return 1
def p(x):
    return 1 + x**2
def j(x, n):
    return x**i * (1 - x**2) if x==0 else 0

In [82]:
def get_y(n):
    bases = get_bases(n)
    parameters = get_parameters(n)
    y = bases[0]
    for i in range(n):
        y += parameters[i]*bases[i+1]
    return y

In [83]:
def get_Y(n):
    y = get_y(n) 
    return u * sympy.diff(y, x, 2) + p * y + 1

## Метод коллокаций
На отрезке $[-1,1]$ выбираем $n-1$ точек $a<x_{0}<x_{1}<...<x_{n-2}<b$ (точки коллокции). Возьмем $x_{i}=-1+(i+1){2\over n}$ и приравняем невязку в этих точках к $0$:
$$
\begin{cases}
Y(x_{0}, a_{1}, ..., a_{n-1}) = 0,
\\
...
\\
Y(x_{n-2}, a_{1}, ..., a_{n-1}) = 0
\end{cases}
=>
M*A = B,где
\begin{cases}
M_{i,0}= -2u(x_{i})+ x_{i}(1-x_{i})p(x_{i}), &i\in[0,n-2],
\\
M_{i,j}= u(x_{i})x_{i}^{i-2}((i-1)i-i(i+1)x_{i}) + p(x_{i})x_{i}^{i}(1-x_{i}), &i\in[0,n-2],j\in[1,n-2]
\\
B_{i} = -1
\end{cases}
$$
Решая полученую систему найдем $a_{0}, ..., a_{n-1}$, а значит и преблеженное решение $y_{n}(x)$

In [175]:
def collacation_method(n):
    x = [-1 + (i+1)*2/(n) for i in range (n-1)]
    matrix = numpy.zeros(shape=(n-1, n-1))
    B = numpy.matrix([-1]*(n-1)).T
    for i in range(n-1):
        matrix[i][0] = -2 * u(x[i]) + x[i] * (1 - x[i]) * p(x[i])
    for i in range(n-1):
        for j in range(1,n-1):
            matrix[i][j] = u(x[i]) * (x[i]**(i-2))*((i-1)*i-i*(i+1)*x[i]) +  p(x[i])*(x[i]**i)*(1 - x[i])
#     return numpy.linalg.solve(matrix, B)
    return matrix, B

In [176]:
collacation_method(5)

(array([[-3.3056  ,  2.176   ,  2.176   ,  2.176   ],
        [-2.2496  , -2.2496  , -2.2496  , -2.2496  ],
        [-1.8336  ,  0.83328 ,  0.83328 ,  0.83328 ],
        [-1.6736  , -0.602496, -0.602496, -0.602496]]), matrix([[-1],
         [-1],
         [-1],
         [-1]]))

## Дискретный МНК

На отрезке $[-1,1]$ выбираем $N$ точек $a<x_0<x_1<...<x_N<b$

Минимизируем интеграл:
$$
S = \sum_{i=1}^{N} Y^2(x_i, a_1, ..., a_{n-1})dx
$$

Для этого решим систему:

$$
\begin{cases}  
\frac{dS}{d a_1} = 2 \int_{-1}^1 Y(x, a_1, ..., a_{n-1}) \frac{d Y(x, a_1, ..., a_{n-1})}{da_1}dx = 0,
\\
...
\\
\frac{dS}{d a_{n-1}} = 2 \int_{-1}^1 Y(x, a_1, ..., a_{n-1}) \frac{d Y(x, a_1, ..., a_{n-1})}{d a_{n-1}}dx = 0
\end{cases} 
$$

In [87]:
def integral_method(n):
    system = [
        sp.integrate(psi * psi.diff(ai), (x, A ,B)) for ai in coefs
    ]
    return sp.solve(system, coefs)

In [88]:
[sympy.integrate(psi * psi.diff(ai), (x, A ,B)) for ai in coefs]

NameError: name 'coefs' is not defined